In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import scipy
import numpy as np
import ijson
from timeit import default_timer as timer
import datetime

In [22]:
businessDf = pd.read_json("yelp_dataset/business.json", lines=True)
businessDf.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [2]:
checkinDf = pd.read_json("yelp_dataset/checkin.json", lines=True)
checkinDf.head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [3]:
tipDf = pd.read_json("yelp_dataset/tip.json", lines=True)
tipDf.head()

,business_id,compliment_count,date,text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ


In [4]:
photoDf = pd.read_json("yelp_dataset/photo.json", lines=True)
photoDf.head()

,business_id,caption,label,photo_id
0,rcaPajgKOJC2vo_l3xa42A,,inside,MllA1nNpcp1kDteVg6OGUw
1,Kn23LDd740SBVJ7mum0fwg,,inside,YjxBE88Bf6CmTEF2LP1UNA
2,ZkGDCVKSdf8m76cnnalL-A,,food,1f7izSjM0WjkDRIVbPy1yw
3,bF8gv7k_rwZtiDLP2ZB04w,,inside,NcSlcDTEEeOaixotOPk-rA
4,50Anorn0DJXFhBr9a9_gHQ,,inside,5IiIo5UKEW0lWqZ6sWrY_A


In [33]:
# Función para limpiar el dataset de usuarios. La limpia consiste en que tenga el formato correcto de json
# para que pueda ser leído, es decir, que empiece y termine con corchetes y que entre cada elemento haya una coma.

import ijson

def cleanDataSet(file):
    startTime = timer()
    
    fileName = file + ".json"
    lastline = None
    with open(fileName,"r", encoding="utf-8") as f:
        lineList = f.readlines()
        lastline=lineList[-1]

    with open(fileName,"r", encoding="utf-8") as f, open(file+"Clean.json","w", encoding="utf-8") as g:
        for i,line in enumerate(f,0):
            if i == 0:
                line = "["+str(line)+","
                g.write(line)
            elif line == lastline:            
                g.write(line)
                g.write("]")
            else:
                line = str(line)+","
                g.write(line)
                
    print(f"**** FINISHED CLEANING {fileName} ****")

In [ ]:
cleanDataSet("yelp_dataset/user")

In [40]:
userUserIds = []
userNames = []
reviewCount = []
yelpingSince = []
usefulUser = []
funnyUser = []
coolUser = []
elite = []
friendCount = []
fans = []
averageStars = []

start = timer()
with open('yelp_dataset/userClean.json', encoding="utf-8") as file:
    parser = ijson.parse(file)
    for prefix, event, value in parser:
        if prefix == "item.user_id":
            userUserIds.append(value)
        elif prefix == "item.name":
            userNames.append(value)
        elif prefix == "item.review_count":
            reviewCount.append(value)
        elif prefix == "item.yelping_since":
            yelpingSince.append(value)
        elif prefix == "item.useful":
            usefulUser.append(value)
        elif prefix == "item.funny":
            funnyUser.append(value)
        elif prefix == "item.cool":
            coolUser.append(value)
        elif prefix == "item.elite":
            elite.append(value)
        elif prefix == "item.friends":
            friendCount.append(len(value.split(", ")))
        elif prefix == "item.fans":
            fans.append(value)
        elif prefix == "item.average_stars":
            averageStars.append(value)

end = timer()
print(f"*** FINISHED READING DATASET IN {end - start} SECONDS ***")

*** FINISHED READING DATASET IN 557.0803704152495 SECONDS ***


In [41]:
userDf = {
    "User Id" : userUserIds,
    "Name" : userNames,
    "Review Count" : reviewCount,
    "Yelping Since" : yelpingSince,
    "Useful" : usefulUser,
    "Funny" : funnyUser,
    "Cool" : coolUser,
    "Elite" : elite,
    "Friend Count" : friendCount,
    "Fans" : fans,
    "Avg Stars" : averageStars
}

userDf = pd.DataFrame(userDf)
userDf.head()

,User Id,Name,Review Count,Yelping Since,Useful,Funny,Cool,Elite,Friend Count,Fans,Avg Stars
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017",99,5,4.03
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,1152,4,3.63
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,28,8,10,,15,0,3.71
3,dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,30,4,14,,525,5,4.85
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018",231,39,4.08


In [49]:
userDf.to_csv("yelp_dataset/userDataFrame.csv")

In [32]:
cleanDataSet("yelp_dataset/review")

**** FINISHED CLEANING yelp_dataset/review.json in 185.80961149975735 SECONDS ****


In [47]:
reviewIds = []
userIds = []
businessIds = []
stars = []
useful = []
funny = []
cool = []
date = []

start = timer()
with open('yelp_dataset/reviewClean.json', encoding="utf-8") as file:
    parser = ijson.parse(file)
    for prefix, event, value in parser:
        if prefix == "item.review_id":
            reviewIds.append(value)
        elif prefix == "item.user_id":
            userIds.append(value)
        elif prefix == "item.business_id":
            businessIds.append(value)
        elif prefix == "item.stars":
            stars.append(value)
        elif prefix == "item.useful":
            useful.append(value)
        elif prefix == "item.funny":
            funny.append(value)
        elif prefix == "item.cool":
            cool.append(value)
        elif prefix == "item.date":
            d = value
            date.append(datetime.datetime.strptime(value, "%Y-%m-%d %H:%M:%S"))

end = timer()
print(f"*** FINISHED READING DATASET IN {end - start} SECONDS ***")

*** FINISHED READING DATASET IN 1007.8703000493224 SECONDS ***


In [50]:
reviewDf = {
    "Review Id" : reviewIds,
    "User Id" : userIds,
    "Business Id" : businessIds,
    "Stars" : stars,
    "Useful" : useful,
    "Funny" : funny,
    "Cool" : cool,
    "Date" : date
}

reviewDf = pd.DataFrame(reviewDf)
reviewDf.head()

,Review Id,User Id,Business Id,Stars,Useful,Funny,Cool,Date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,2018-01-30 23:07:38


In [51]:
reviewDf.to_csv("yelp_dataset/reviewDataFrame.csv")